<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/JARVIS_ML_LightGBM_GPUvsCPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook shows how GPU can accelerate model training with respect to CPU using JARVIS-ML CFID dataset .

##Download LightGBM

In [1]:
! git clone --recursive https://github.com/Microsoft/LightGBM


Cloning into 'LightGBM'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18246 (delta 3), reused 2 (delta 1), pack-reused 18229
Receiving objects: 100% (18246/18246), 12.25 MiB | 12.13 MiB/s, done.
Resolving deltas: 100% (13319/13319), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 8.51 MiB | 3.02 MiB/s, done.
Resolving deltas: 100% (17565/17565), done.
Submodule path 'compute': checked out '36c89134d4013b2e5e45bc55656a18bd6141995a'


## Install LightGBM with GPU
## Make sure you have enabled GPU using: " Edit>Notebook Settings>GPU " from the dropdown menu up

In [2]:

! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_2_2
-- Looking for CL_VERSION_2_2 - found
-- Found OpenCL: /usr/lib/x86_64-linux-gnu/libOpenCL.so (found version "2.2") 
-- OpenCL include directory: /usr/include
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--

##GPU information

In [7]:
!nvidia-smi

Sat Jun 27 16:29:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##Install jarvis-tools

In [17]:
!pip install jarvis-tools

     |████████████████████████████████| 798kB 8.7MB/s 
     |████████████████████████████████| 296kB 30.0MB/s 


##Get example data from JARVIS

In [18]:
from jarvis.ai.pkgs.utils import get_ml_data
from jarvis.ai.pkgs.utils import regr_scores
X,y,jid=get_ml_data()
#Formation energy for 3D materials, you can choose other properties/dataset as well

##Formation energy model training with GPU
### Other hyperparameters for materials properties are available at https://github.com/usnistgov/jarvis/tree/master/jarvis/ai/pkgs/lgbm

In [27]:
%%time
import lightgbm as lgb
from sklearn.model_selection import train_test_split
lgbm = lgb.LGBMRegressor(device= 'gpu',n_estimators= 1170,learning_rate= 0.15375236057119931,num_leaves= 273)
           
X_train, X_test, y_train, y_test, jid_train, jid_test = train_test_split(X, y, jid, random_state=1, test_size=.1)
lgbm.fit(X_train,y_train)
pred = lgbm.predict(X_test)
reg_sc = regr_scores(y_test, pred)
print (reg_sc['mae'])

0.12625287443194153
CPU times: user 34min 31s, sys: 5.08 s, total: 34min 36s
Wall time: 17min 38s


##Accuracy of the model can be further improved by increasing number of trees, decreasing learning rate, and increasing number of leaves. However, it requires more training time and the model may overfit. You can skip this step, if you just want to see the speed up

In [28]:
%%time
import lightgbm as lgb
from sklearn.model_selection import train_test_split
lgbm = lgb.LGBMRegressor(device= 'gpu',n_estimators= 2170,learning_rate= 0.05375236057119931,num_leaves= 573)
           
X_train, X_test, y_train, y_test, jid_train, jid_test = train_test_split(X, y, jid, random_state=1, test_size=.1)
lgbm.fit(X_train,y_train)
pred = lgbm.predict(X_test)
reg_sc = regr_scores(y_test, pred)
print (reg_sc['mae'])

0.11970056899179356
CPU times: user 2h 11min 2s, sys: 19.8 s, total: 2h 11min 22s
Wall time: 1h 6min 52s


## Formation energy model training with CPU

In [29]:
%%time
import lightgbm as lgb
from jarvis.ai.pkgs.utils import regr_scores
from sklearn.model_selection import train_test_split
lgbm = lgb.LGBMRegressor(n_estimators= 1170,learning_rate= 0.15375236057119931,num_leaves= 273, device= 'cpu')
X_train, X_test, y_train, y_test, jid_train, jid_test = train_test_split(X, y, y, random_state=1, test_size=.1)
lgbm.fit(X_train,y_train)
pred = lgbm.predict(X_test)
reg_sc = regr_scores(y_test, pred)
print (reg_sc['mae'])

0.1263575987673102
CPU times: user 46min 10s, sys: 4.53 s, total: 46min 15s
Wall time: 23min 40s


## Comparing wall time for GPU (17 mins 18 s = 1038 s ) and wall time for CPU (23 mins 40 s = 1420 s), we accelerated by ~27 %

In [31]:
#GPU (17 mins 18 s = 1038 s )
#CPU (23 mins 40 s = 1420 s)
perc_diff = 100*(1420-1038)/1420
print (perc_diff)


26.901408450704224
